# Business Analytics II – Project II
### 2020120083 손영진

## Overview
This project involves building classification models to predict whether an Airbnb host is a "Superhost".
The process includes:
1. Data Preprocessing (Handling missing values, Encoding, Balancing, Scaling)
2. Model Building & Evaluation (Logistic Regression, Decision Tree, Random Forest, MLP, KNN, Naive Bayes)
3. Hyperparameter Tuning
4. Final Prediction on Test Data

# 1. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# 2. Data Load

In [ ]:
# Load datasets
train_path = 'train_data.csv'
test_path = 'test_f25.xlsx'

df_train = pd.read_csv(train_path)
df_test = pd.read_excel(test_path)

print(f"Train Shape: {df_train.shape}")
print(f"Test Shape: {df_test.shape}")
df_train.head()

# 3. Data Preprocessing

In [ ]:
# Target Variable
target_col = 'host_is_superhost'

# Convert target to binary (t/f -> 1/0)
df_train[target_col] = df_train[target_col].map({'t': 1, 'f': 0})

# Check for missing values in target and drop them
df_train = df_train.dropna(subset=[target_col])
print(f"Train Shape after dropping missing targets: {df_train.shape}")

# Separate Target and Features
y = df_train[target_col]
X = df_train.drop(columns=[target_col])

# Identify categorical and numerical columns
cat_cols = X.select_dtypes(include=['object']).columns
num_cols = X.select_dtypes(include=['int64', 'float64']).columns

print(f"Categorical Columns: {len(cat_cols)}")
print(f"Numerical Columns: {len(num_cols)}")

In [ ]:
# Handle Missing Values
# For numerical, fill with median
for col in num_cols:
    X[col] = X[col].fillna(X[col].median())
    if col in df_test.columns:
        df_test[col] = df_test[col].fillna(df_test[col].median())

# For categorical, fill with mode
for col in cat_cols:
    X[col] = X[col].fillna(X[col].mode()[0])
    if col in df_test.columns:
        df_test[col] = df_test[col].fillna(df_test[col].mode()[0])

In [ ]:
# One-Hot Encoding
X_encoded = pd.get_dummies(X, columns=cat_cols, drop_first=True)
df_test_encoded = pd.get_dummies(df_test, columns=cat_cols, drop_first=True)

# Align columns (Ensure train and test have same features)
X_encoded, df_test_encoded = X_encoded.align(df_test_encoded, join='left', axis=1, fill_value=0)

print(f"Encoded Train Features Shape: {X_encoded.shape}")
print(f"Encoded Test Features Shape: {df_test_encoded.shape}")

# 4. Data Balancing (Upsampling)

In [ ]:
# Combine X and y for resampling
train_data = pd.concat([X_encoded, y], axis=1)

# Check class distribution
print("Original Class Distribution:")
print(y.value_counts())

# Separate majority and minority classes
df_majority = train_data[train_data[target_col] == 0]
df_minority = train_data[train_data[target_col] == 1]

# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=len(df_majority),    # to match majority class
                                 random_state=42) # reproducible results

# Combine majority class with upsampled minority class
df_balanced = pd.concat([df_majority, df_minority_upsampled])

# Display new class counts
print("\nBalanced Class Distribution:")
print(df_balanced[target_col].value_counts())

# Separate X and y again
X_balanced = df_balanced.drop(columns=[target_col])
y_balanced = df_balanced[target_col]

# 5. Feature Scaling

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_balanced)
X_test_scaled = scaler.transform(df_test_encoded)

X_scaled = pd.DataFrame(X_scaled, columns=X_balanced.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=df_test_encoded.columns)

# 6. Model Building & Evaluation (5-Fold CV)

In [ ]:
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "MLP (Neural Network)": MLPClassifier(max_iter=1000, random_state=42),
    "KNN": KNeighborsClassifier(),
    "Naive Bayes": GaussianNB()
}

results = {}

print("--- 5-Fold Cross Validation Results ---")
for name, model in models.items():
    scores = cross_val_score(model, X_scaled, y_balanced, cv=5, scoring='accuracy')
    results[name] = np.mean(scores)
    print(f"{name}: Mean Accuracy = {np.mean(scores):.4f}")

# 7. Hyperparameter Tuning (Random Forest)

In [ ]:
# Tuning Random Forest as it often performs well
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=1)
grid_search.fit(X_scaled, y_balanced)

best_rf = grid_search.best_estimator_
print(f"\nBest Parameters for Random Forest: {grid_search.best_params_}")
print(f"Best Cross-Validation Score: {grid_search.best_score_:.4f}")

# 8. Final Prediction

In [ ]:
# Predict on Test Data using the best model
final_predictions = best_rf.predict(X_test_scaled)

# Create submission dataframe
submission = pd.DataFrame({
    'No': df_test['No'],  # Assuming 'No' is the identifier in test data
    'host_is_superhost_pred': final_predictions
})

# Map 1/0 back to t/f if needed, or keep as binary. Instructions say "predict which hosts are likely to be Superhosts"
# Usually binary is fine, but let's check format. Keeping as 1/0 for now or mapping back.
submission['host_is_superhost_pred'] = submission['host_is_superhost_pred'].map({1: 't', 0: 'f'})

submission.head()

In [ ]:
# Save to Excel
submission.to_excel('prediction_result.xlsx', index=False)
print("Prediction saved to prediction_result.xlsx")

# 9. Summary

**Model Building Process:**
1.  **Data Preprocessing**: 
    -   Target variable `host_is_superhost` was converted to binary.
    -   Missing values were filled (median for numerical, mode for categorical).
    -   Categorical variables were One-Hot Encoded.
    -   Train and Test features were aligned to ensure consistency.
    -   Data was balanced using upsampling to address class imbalance.
    -   Features were scaled using StandardScaler.

2.  **Model Evaluation**:
    -   Various models (Logistic Regression, Decision Tree, Random Forest, MLP, KNN, Naive Bayes) were evaluated using 5-fold cross-validation.
    -   Random Forest typically showed strong performance.

3.  **Tuning**:
    -   GridSearchCV was used to tune Random Forest hyperparameters.

4.  **Prediction**:
    -   The best performing model was used to predict `host_is_superhost` for the test dataset.
    -   Results are saved in `prediction_result.xlsx`.